# **Latar Belakang**

Jadi pada era digital ini, banyak sekali anak-anak yang sudah diberikan gadget oleh orang tuanya, berawal dari fenomena inilah akhirnya kita mengambil sebuah langkah untuk mengenalkan manfaat dari buah-buahan kepada anak-anak tersebut, terutama anak sd yang sudah mulai gemar membaca.

Disini kami membuat sebuah aplikasi yang bernama Buah-Seru, dengan aplikasi ini anak-anak dapat mendeteksi buah dan setelah dideteksi akan muncul sebuah kuis yang akan diberikan kepada anak tersebut, misalnya anak tersebut mendeteksi apel maka akan tampil tiga pilihan, apel, jeruk, pisang, dan anak akan disuruh memilih dari ketiga opsi tersebut, benar maupun salah jawaban anak tersebut tetap akan diberikan info mengenai buah yang telah di scan tersebut.

Dengan adanya aplikasi ini kami harap pemberian gadget oleh orangtua kepada anaknya tidak hanya digunakan untuk bermain game, melainkan digunakan juga untuk melakukan pembelajaran interaktif dan juga meningkatkan literasi dari anak tersebut melalui aplikasi Buah-Seru ini.

# Persiapan dataset sebelum di normalisasi
Disini kami menggunakan dataset yang berjumlah 6600 gambar, masing-masing gambar yaitu gambar apel, pisang, dan jeruk terdiri dari 2200 dataset untuk setiap kelas tersebut.

Karena awal gambar yang didapatkan ketika mengunduh dataset masih memiliki background akhirnya digunakanlah teknik grabcut yang berguna untuk mengeliminasi background sehingga yang tersisa akhrinya hanya datasetnya saja, dan gambar latar belakangnya hilang.

In [ ]:
import os
import cv2
import numpy as np

# menentukan folder sumber dan tujuan
sumber = '../data4/img/train/apel2'
tujuan = '../output/apel2'

# melakukan looping secara rekursif melalui semua subfolder
for root, _, files in os.walk(sumber):
    for fname in files:
        if not fname.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        # path atau jalur
        src_path = os.path.join(root, fname)
        rel_path = os.path.relpath(src_path, sumber)
        dest_path = os.path.join(tujuan, rel_path)

        # membuat folder tujuan jika belum ada
        os.makedirs(os.path.dirname(dest_path), exist_ok=True)

        # membaca gambar
        img = cv2.imread(src_path)
        h, w = img.shape[:2]

        # mendefinisikan rectangle awal (contoh: margin 10% dari tepi)
        x = int(w * 0.1)
        y = int(h * 0.1)
        rw = int(w * 0.8)
        rh = int(h * 0.8)
        rect = (x, y, rw, rh)

        # melakukan inisialisasi mask & model
        mask = np.zeros((h, w), np.uint8)
        bgdModel = np.zeros((1,65), np.float64)
        fgdModel = np.zeros((1,65), np.float64)

        # menjalankan GrabCut
        cv2.grabCut(img, mask, rect, bgdModel, fgdModel, iterCount=5, mode=cv2.GC_INIT_WITH_RECT)

        # membuat mask biner: 0 itu untuk background, 1 itu untuk foreground
        mask2 = np.where((mask==2)|(mask==0), 0, 1).astype('uint8')

        # menerapkan mask ke gambar
        segmented = img * mask2[:, :, np.newaxis]

        # menyimpan hasil
        cv2.imwrite(dest_path, segmented)
        print(f'Processed: {rel_path}')


# Kode - CNN

Disini kami menggunakan CNN karena datasetnya berupa gambar, selain itu CNN adalah salah satu algoritma Neural Network yang tersedia di tensorflow lite sehingga memudahkan kita untuk melakukan pengembangan aplikasi mobile tanpa harus melakukan compile di cloud ataupun perangkat lainnya.

## Mengimport library yang dibutuhkan

In [ ]:
import os
# digunaka untuk operasi sistem file seperti mencari path file
import glob
# ini digunakan untuk mencari file dengan format tertentu jpg misalnya
import random
# ini digunakan untuk mengacak data
import numpy as np
# digunakan untuk operasi array seperti vector dan matirks
from PIL import Image
# digunakan untuk memanipulasi gambar di python seperti merubah ukuran dsb
import tensorflow as tf
# framework machine learning yang digunakan
from tensorflow.keras import layers, models, callbacks
# mengimpor sub modul dari keras yaitu API yang ada di tensorflow untuk membuat layer, model ,dan callback(penjaga saat training)
from sklearn.model_selection import train_test_split
# untuk melakukan random split dan stratified split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
# untuk menghitung dan menampilkan confussion matrix
import matplotlib.pyplot as plt
#  untuk plotting confussion matrix
from collections import Counter
# menghitung frekuensi label

## Path untuk folder dataset & parameter

In [ ]:
TEMPAT_DATA_TRAIN    = '../data5'
UKURAN_BATCH         = 32
EPOCHS               = 10
LEARNING_RATE        = 1e-3
UKURAN_GAMBAR        = (258, 320)
VAL_SPLIT            = 0.2
TEST_SPLIT           = 0.1
SEED                 = 42

### Penjelasan
**TEMPAT_DATA_TRAIN =** '../data5' menunjuk ke folder di mana dataset tersimpan.

**UKURAN_BATCH**       = 32 yaitu jumlah  sampel yang diproses dalam satu batch saat melakukan training (jumlah data yang dikirimkan untuk sekali proses training).

**EPOCHS**               = 10 ini adalah nilai iterasi yang dilakukan oleh model untuk melakukan training, jadi batch tadi ada hubungannya dengan epoch, data yang akan di training tadi sudah dibagi ke dalam batch kemudian akan di lakukan proses training atau latih untuk model.

**LEARNING_RATE**        = 1e-3 ini adalah nilai dari kecepatan pembelajaran yang digunakan oleh sebuah optimizer untuk mengubah bobot model pada setiap iterasi. Optimizer yang digunakan di sini adalah Adam atau adaptive moment estimation yaitu salah satu jenis optimizer dengan memanfaatkan gradien untuk menyesuaikan pembaruan bobot secara adaptif.


**UKURAN_GAMBAR**        = (258, 320) ini adalah ukuran dari gambar untuk dilakukan resize setiap gambar sebelum akhirnya dimasukkan ke dalam model, tingginya 258 dan lebarnya 320.

**VAL_SPLIT**            = 0.2 pemisahan data untuk validation, validation disini berguna untuk memantau performa model tiap epoch tanpa mempengaruhi test set.

**TEST_SPLIT**           = 0.1 pemisahan data untu test, test ini dilakukan sekali saja untuk menilai performa akhir dari model tersebut.

**SEED**                = 42 digunakan untuk melakukan pengacakan pada data.

## Mengumpulkan file dan label

In [ ]:
semua_file  = glob.glob(os.path.join(TEMPAT_DATA_TRAIN, '*', '*'))
semua_label = [os.path.basename(os.path.dirname(f)) for f in semua_file]
nama_kelas = sorted(set(semua_label))
label_ke_index = {c:i for i,c in enumerate(nama_kelas)}
list_index    = [label_ke_index[l] for l in semua_label]

print("Total sampel:", len(semua_file))
print("Kelas:", nama_kelas)
print("Jumlah sampel per kelas:", Counter(semua_label))

### Penjelasan
Disini awalnya kita akan menggunakan modul glob untuk mencari semua file yang ada di dalam folder TEMPAT_DATA_TRAIN, setelah itu akan dilakukan Loop untuk setiap path atau jalur f di semua_label, setelah itu nama kelas akan diurutkan secara alfabetis menggunakan sorted, selanjutnya buat dictionary untuk memetakan setiap nama kelas ke sebuah indeks integer, lakukan loop lagi untuk setiap nama kelas l di semua_label untuk mengganti nama kelas tersebut dengan indeks numeriknya menurut data yang ada pada variabel lakel_ke_index.

## Menghitung ukuran pemisahan data

In [ ]:
n_total = len(semua_file)
n_test  = int(n_total * TEST_SPLIT)
n_val   = int(n_total * VAL_SPLIT)
print(f"\nPemisahan data yang akan dilakukan: Test={n_test} samples, Val={n_val} samples, Train={n_total - n_test - n_val} samples")

### Penjelasan
**n_test**  = int(n_total * TEST_SPLIT) ini itu berarti dataset untuk testnya itu 10% dari total

**n_val**   = int(n_total * VAL_SPLIT) ni itu berarti dataset untuk validationnya itu 20% dari total

## Melakukan stratified split dengan ukuran tetap

In [ ]:
file_file  = np.array(semua_file)
label_label = np.array(list_index)

# a) ambil test set
f_temp, f_test, l_temp, l_test = train_test_split(
    file_file, label_label,
    test_size=n_test,
    shuffle=True,
    stratify=label_label,
    random_state=SEED
)

# b) dari sisa ambil val set
f_train, f_val, l_train, l_val = train_test_split(
    f_temp, l_temp,
    test_size=n_val,
    shuffle=True,
    stratify=l_temp,
    random_state=SEED
)

train_file_file, train_idxs = f_train.tolist(), l_train.tolist()
val_file_file,   val_idxs   = f_val.tolist(),   l_val.tolist()
test_file_file,  test_idxs  = f_test.tolist(),  l_test.tolist()

print("\nSplit distributions:")
print(" Train:", Counter([nama_kelas[i] for i in train_idxs]))
print(" Val  :", Counter([nama_kelas[i] for i in val_idxs]))
print(" Test :", Counter([nama_kelas[i] for i in test_idxs]))

NameError: name 'np' is not defined

### Penjelasan
Pada bagian ini kode akan mengubah daftar dari path file dan daftar indeks label menjadi array numpy pada variabel **file_file** dan juga **label_label** agar dapat diproses oleh t**rain_test_split** nantinya. Disini kita akan membagi data menjadi dua bagian terlebih dahulu yaitu ada **n_test** dan sisa data yaitu **f_temp** dan **l_temp** yang pada kondisi ini belum terpakai. Selanjutnya yaitu pemilihan sampel untuk test set yang dilakukan secara acak tetapi tetap mempertahankan proporsi dari setiap kelas karena ada parameter **stratify=label_label** sehingga membuat label di test set akan terdistribusi secara merata.

Sisa data yang belum terpakai tadi yaitu **f_temp** dan **l_temp** akan dipisah lagi menjadi validation set sebanyak **n_val** sampel dan sisanya digunakan untuk training set. Proses ini membuat stratifikasi label agar nantinya validation setnya itu memiliki komposisi yang seimbang. Setelah pemgbagian selesai dilakukan, semua array hasil split diubah kembali menjadi list python yaitu **train_file_file**, **val_file_file**, dan **test_file_file** untuk pathnya serta **train_idxs**, **val_idxs**, dan **test_idxs** untuk labelnya. Kalau dilihat disitu ada counter yang berguna untuk memverifikasi bahwa data train, validation, dan testnya sudah pas proporsinya.

## Membuat tf.data.Dataset dengan melakukan preprocessing manual minmax

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

def load_and_preprocess(path, label):

    def _py_load(path_str):
        img = Image.open(path_str.decode('utf-8')).convert('RGB')

        try:
            resample_method = Image.Resampling.LANCZOS
        except AttributeError:
            resample_method = Image.LANCZOS
        img = img.resize((UKURAN_GAMBAR [1], UKURAN_GAMBAR [0]), resample_method)
        arr = np.array(img).astype(np.float32) / 255.0
        return arr



    img = tf.numpy_function(func=_py_load, inp=[path], Tout=tf.float32)

    img.set_shape((UKURAN_GAMBAR [0], UKURAN_GAMBAR [1], 3))
    one_hot = tf.one_hot(label, depth=len(nama_kelas))
    return img, one_hot

def prepare(file_file, label_label, shuffle=False, augment=False):
    ds = tf.data.Dataset.from_tensor_slices((file_file, label_label))
    if shuffle:
        ds = ds.shuffle(buffer_size=1000, seed=SEED)
    ds = ds.map(load_and_preprocess, num_parallel_calls=AUTOTUNE)
    if augment:
        aug = tf.keras.Sequential([
            layers.RandomFlip('horizontal'),
            layers.RandomRotation(0.1),
            layers.RandomZoom(0.1),
        ])
        ds = ds.map(lambda x, y: (aug(x), y), num_parallel_calls=AUTOTUNE)
    return ds.batch(UKURAN_BATCH).prefetch(AUTOTUNE)

train_ds = prepare(train_file_file, train_idxs, shuffle=True,  augment=True)
val_ds   = prepare(val_file_file,   val_idxs,   shuffle=False, augment=False)
test_ds  = prepare(test_file_file,  test_idxs,  shuffle=False, augment=False)

### Penjelasan
Di bagian kode ini disini kita membuat sebuah pipeline data (sebuah rangkaian dari tahapan yang ada pada proses di machine learing untuk menyiapkan data dari bentuk mentah hingga siap digunakan oleh model) menggunakan API tf,data agar pembacaan dan pemrosesan gambar dapat berjalan secara efisien dan terintegrasi langsung ke dalam alur TensorFlow. Awalnya konstanta dari AUTOTUNE=tf.data.AUTOTUNE menginstruksi tensorflow untuk menyesuaikan sendiri tingkat paralelisme pemanggilan fungsi dan prefetchingnya berdasarkan dari sumber daya yang tersedia.

Selanjutnya fungsi load_and_preprocess(path, label) bertugas untuk memuat setiap file gambar dari disk atau ssd (tergantung penyimpanan pc masing-masing, disini saya menggunakan ssd) dan mengubahnya menjadi tensor siap pakai (disini tensor yang saya maksud adalah struktur data yang merepresentasikan array-multi dimensi), di dalam tensor tersebut ada tf.numpy_function yang akan memanggil fungsi python _py_load yang akan membuka gambar via pillow, lalu gambar akan dikonversi ke format RGB, setelah itu akan dilakukan resize ke dimensinya yaitu berupa (lebar, tinggi) sesuai dengan UKURAN_GAMBAR yang telah didefinisikan di atas dengan metode resampling LANCZOS (yaitu teknik interpolasi berbasi fungsi sinc yang umum digunakan untuk mengubah ukuran gambar dengan kualitas tinggi), meskipun gambarnya beresolusi 258x320 tapi pixelnya disini tetap memiliki rentang dari 0 sampai 255, jadi resolusi tidak masalah, setelah itu pixel akan diubah menjadi array numpy dan akan dilakukan normalisasi min-max melalui pembagian dengan nilai 255, dengan cara ini kita dapat memetakan nilai mulai dari rentang 0.0 sampai 1.0 tanpa mempengaruhi ukuran dari gambar tersebut sehingga nantinya pixel berada pada skala yang sama dan akan mempercepat proses konvergensi model. Tensor hasil normalisasi ini kemudian akan diberi shape atau ukuran secara eksplisit berupa (tinggi, lebar, 3), hal ini dilakukan agar TensorFlow dapat membangun graph dengan benar dan label integer dapat diubah menjadi vector one-hot sepanjang jumlah kelas (len(nama_kelas)).  

Setelah itu fungsi prepare(file_file, label_label, shuffle=False, augment=False) menyusun objek tf.data.Dataset dari sepasang list path dan label, disini jika parameter dari shuffle=True, maka dataset akan diacak dengan buffer size 1000 dan seed yang ditetapkan sebelumnya, selanjutnya setiap elemen dipetakan ke load_and_preprocess, setelah itu jika augment=True maka akan diterapkan  transformasi augmentasi pada datasetnya lewat sebuah model keras. dan selanjutnya dataset dibagi per batch sesuai dengan ukuran dari batch yang telah diaturan sebelumnya lewat variabel UKURAN_BATCH, kemudian akan dilakukan prefetch agar data berikutnya sudah siap ketika GPU selesai memproses batch sebelumnya.

setelah itu, maka tiga dataset akan disiapkan sesuai fungsinya masing masing, traind_ds dengan shuffle=True dan augment=True digunakan untuk training, lalu val_ds dan test_ds dibuat tanpa shuffle dan augmentasi agar performa konsisten dan tidak dipengaruhi oleh variasi augmentasi.


## Membangun dan melakukan compile untuk model

In [ ]:
model = models.Sequential([
    layers.Input(shape=(*UKURAN_GAMBAR , 3)),
    layers.Conv2D(32, 3, activation='relu', padding='same'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu', padding='same'),
    layers.MaxPooling2D(),
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(nama_kelas), activation='softmax'),
], name='buah_manual_preprocess_cnn')

model.compile(
    optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

### Penjelasan
Jadi disini itu karena kita menggunakan deeplearning dan algoritmanya CNN yang ada pada tensorflow kita tidak bisa melakukan import seperti pada library biasanya, misalnya ketika kita menggunakan KNN kita hanya tinggal melakukan import dari scikit-learn, tapi di tensorflow ini kita menggunakan penyusunan layer dari keras untuk menyusun algoritma CNN-nya, jadi layer yang digunakan untuk menyusun model CNN-nya yaitu :

1. Input layer
2. Convolutional layer
3. Max Pooling layer
4. Convolutional layer
5. Max Pooling layer
6. GlobalAveragePooling
7. Fully Connected (Dense) layer
8. Dropout layer
9. Output layer

Prosesnya yaitu input akan menerima citra RGB berukuran 258x320 dan menyiapkan tensor berdimensi (258, 320, 3). Setelah itu, layer konvolusi pertama menerapkan 32 filter berukuran 3x3 dengan padding "same" yang kemudian diaktifkan melalui fungsi relu untuk menambahkan kemampuan belajar pola non-linier. Setelah dilakukannya ektrasi awal maka layer max-pooling pertama akan mengambil nilai maksimum dengan ukuran 2x2 sehingga mereduksi tinggi dan lebar pada fitur, tetapi masih tetap mempertahankan fitur-fitur pentingnya. Proses ekstraksi ini kemudian dilanjutkan oleh layer konvolusi kedua yang menggunakan 64 filter berukuran 3x3 dengan padding "same" dan relu dan diikuti juga oleh lapisan max-pooling kedua untuk menurunkan resolusi spasial lebih lanjut.


Disini itu kita tidak melakukan flatten (suatu layer yang mengubah semua nilai piksel di peta fitur menjadi satu vektor panjang), tapi model ini memakai globalaveragepooling yang menghitung rata rata nilai pada tiap fitur sehingga dapat menghasilkan satu vektor per filter tanpa membuat jumlah dari parameternya menjadi terlalu banyak atau membengkak. Vektor tersebut kemudian akan diproses kembali oleh layer dense yang berukuran 128 neuron dengan aktivasi relu untuk menggabungkan fitur, lalu lapisan dropout akan mematikan 30% neuron secara acak di lapisan ini selama training. Setelah itu layer output akan memiliki jumlah neuron yang sama banyaknya dengan kelas dan juga menggunakan aktivasi softmax untuk mengonversi nilai mentah menjadi distribusi probabilitas prediksi.




## Melakukan callbacks dan training

In [ ]:
callbacks_list = [
    callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    callbacks.ModelCheckpoint('model_terbaik.h5', monitor='val_accuracy', save_best_only=True),
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks_list
)

### Penjelasan
Mekanisme Callback :
Jadi disini ada earlystopping untuk memantau apakah validation loss turun atau tidak selama tiga epoch berturut-turut, kalau turun maka akan dihentikan, disini juga ada kode untuk melakukan perbaikan ketika hal tersebut terjadi yaitu restore_best_weight=True. Ada sebuah checkpoint disini yang berguna untuk menyimpan bobot mode ke file model_terbaik.h5 setiap kali akurasi dari validari (val_accuracy) meningkat.

modelfit digunakan untuk mengatur jumlah epoch, memberikan data train, memberikan data validasi, dan menggunakan callback.

## Melakukan evaluasi pada test set


In [ ]:
model = tf.keras.models.load_model('model_terbaik.h5')
loss, acc = model.evaluate(test_ds)
print(f"\nAkurasi test: {acc:.4f}")

### Penjelasan
TensorFlow akan secara berurutan memberikan setiap batch dari test_ds ke model, menghitung loss (seberapa besar prediksi model berbeda dari label sebenarnya) dan akurasi (persentase prediksi yang benar), lalu mengembalikan nilai rata‐rata dari metrik ini di seluruh test set.

## Confussion matrix untuk evaluasi model

In [ ]:
y_true, y_pred = [], []
for x_batch, y_batch in test_ds:
    probs = model.predict(x_batch, verbose=0)
    y_true.extend(tf.argmax(y_batch, axis=1).numpy())
    y_pred.extend(np.argmax(probs, axis=1))

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=nama_kelas)
fig, ax = plt.subplots(figsize=(8, 8))
disp.plot(ax=ax, xticks_rotation=90)
plt.title('Confusion Matrix pada Test Set')
plt.tight_layout()
plt.show()

### Penjelasan


## Menyimpan model dan melakukan konversi ke tflite

In [ ]:
model.save('siscer_cnn_deteksi_buah.h5')
tflite = tf.lite.TFLiteConverter.from_keras_model(model).convert()
with open('siscer_cnn_deteksi_buah.tflite', 'wb') as f:
    f.write(tflite)

print("Selesai menyimpan model dengan format .h5 & .tflite")

### Penjelasan


## Menyimpan label dari model

In [ ]:
CLASS_FILE = 'class.txt'
with open(CLASS_FILE, 'w') as f:
    for cls in class_names:
        f.write(f"{cls}\n")
print(f"Saved {len(class_names)} labels to {CLASS_FILE}")

### Penjelasan


# Melakukan pengujian terhadap model yang sudah di konversi

In [ ]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf


KERAS_MODEL_PATH  = 'siscer_cnn_deteksi_buah.h5'
TFLITE_MODEL_PATH = 'siscer_cnn_deteksi_buah.tflite'
IMG_SIZE = (320, 258)

with open('class.txt', 'r') as f:
    class_names = [line.strip() for line in f if line.strip()]


def load_and_preprocess_image(path, img_size=IMG_SIZE):
    img = Image.open(path).convert('RGB')
    img = img.resize(img_size)
    arr = np.asarray(img, dtype=np.float32) / 255.0
    return arr

def predict_with_keras(model_path, img_paths):
    model = tf.keras.models.load_model(model_path)
    images = np.stack([load_and_preprocess_image(p) for p in img_paths], axis=0)
    probs  = model.predict(images)
    preds  = np.argmax(probs, axis=1)

    for path, p, prob in zip(img_paths, preds, probs):
        label = class_names[p] if p < len(class_names) else f"Unknown({p})"
        print(f"{os.path.basename(path)} → {label} ({prob[p]*100:.2f}%)")


def predict_with_tflite(model_path, img_paths):
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    input_details  = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    for path in img_paths:
        img = load_and_preprocess_image(path)[None, ...].astype(np.float32)
        interpreter.set_tensor(input_details[0]['index'], img)
        interpreter.invoke()
        probs = interpreter.get_tensor(output_details[0]['index'])[0]
        pred  = np.argmax(probs)
        label = class_names[pred] if pred < len(class_names) else f"Unknown({pred})"
        print(f"{os.path.basename(path)} → {label} ({probs[pred]*100:.2f}%)")


if __name__ == '__main__':

    test_images = [
        '../coba/apel3.jpg',
        '../coba/pisang.jpg',
    ]

    print("=== Hasil perkiran Keras .h5 ===")
    predict_with_keras(KERAS_MODEL_PATH, test_images)

    print("\n=== Hasil perkiraan TFLite .tflite ===")
    predict_with_tflite(TFLITE_MODEL_PATH, test_images)
